# ADIA Refactored – Getting Started

This notebook gives a gentle intro to working with the `adia_refactored` package from a notebook environment. It covers:

- Ensuring your environment can import the package
- A quick single-series demo
- A small batch demo using Parquet files
- (Optional) Running the included example script

If you see import errors for `numpy`/`pandas`/`pyarrow`, run the optional install cell below and then restart the kernel.


In [ ]:
# Optional: install project dependencies into the active kernel
# If imports fail later, run this, then restart the kernel and re-run
import sys
import subprocess

def ensure_deps():
    try:
        import numpy  # noqa: F401
        import pandas  # noqa: F401
        import pyarrow  # noqa: F401
        print("Dependencies appear to be installed. Skipping install.")
        return
    except Exception:
        pass
    print("Installing requirements...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])  # nosec

ensure_deps()


In [ ]:
# Setup: make package importable and validate config
import sys, pathlib

project_root = pathlib.Path().resolve()
parent_dir = project_root.parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from adia_refactored import compute_predictors_for_values, run_batch, validate_config
validate_config()
print("Imports OK and config validated.")


In [ ]:
# Single-series demo
import numpy as np

np.random.seed(42)
n = 120
bp = 60
values = np.concatenate([
    np.random.normal(0, 1.0, bp),
    np.random.normal(1.5, 1.5, n - bp),
])
periods = np.concatenate([np.zeros(bp), np.ones(n - bp)])

preds, meta = compute_predictors_for_values(values, periods, B_boot=20, energy_enable=False)
preds, meta


In [ ]:
# Batch demo (writes and reads Parquet files under _tmp_notebook)
from pathlib import Path
import numpy as np
import pandas as pd

# Prepare synthetic batch
tmp_dir = Path("_tmp_notebook"); tmp_dir.mkdir(exist_ok=True)
frames = []
for i in range(4):
    np.random.seed(100 + i)
    n = 80; bp = 40
    if i % 2 == 0:
        vals = np.concatenate([np.random.normal(0, 1.0, bp),
                               np.random.normal(1.2, 1.0, n - bp)])
    else:
        vals = np.concatenate([np.random.normal(0, 0.8, bp),
                               np.random.normal(0, 1.8, n - bp)])
    periods = np.concatenate([np.zeros(bp), np.ones(n - bp)])
    df = pd.DataFrame({"value": vals, "period": periods})
    df.index = pd.MultiIndex.from_tuples([(f"series_{i}", t) for t in range(n)], names=["id", "time"])
    frames.append(df)

all_df = pd.concat(frames).sort_index()

in_path = tmp_dir / "sample_batch.parquet"
out_pred = tmp_dir / "sample_predictors.parquet"
out_meta = tmp_dir / "sample_metadata.parquet"

all_df.to_parquet(in_path)

pred_df, meta_df = run_batch(
    input_parquet=str(in_path),
    out_pred_parquet=str(out_pred),
    out_meta_parquet=str(out_meta),
    B_boot=20,
    energy_enable=False,
    n_jobs=1,
    verbose=False,
)

print(f"Processed series: {len(pred_df)}")
print(f"Predictor columns: {len(pred_df.columns)}")
pred_df.head(), meta_df.head()


In [ ]:
# Optional: run the included script and (optionally) clean up temp files
print("Running example_usage.py ...")
%run example_usage.py

# Optional cleanup for this notebook's temp files
from pathlib import Path
for p in Path("_tmp_notebook").glob("*.parquet"):
    try:
        p.unlink()
    except Exception:
        pass
